In [1]:
#!pip install gym_super_mario_bros
#!pip install nes_py
#!pip install stable_baselines
#!pip install baselines
#%tensorflow_version 1.x
#!pip install tensorflow==1.15.0 --user
import tensorflow as tf
import time
import numpy as np
from nes_py.wrappers import JoypadSpace
import gym_super_mario_bros

from wrappers import wrapper
from gym_super_mario_bros.actions import RIGHT_ONLY
#from stable_baselines.ddpg.policies import CnnPolicy
from stable_baselines.common.policies import MlpLstmPolicy, CnnLstmPolicy, MlpPolicy
from stable_baselines.common.vec_env import VecVideoRecorder, SubprocVecEnv, DummyVecEnv
from stable_baselines import A2C, PPO2
from stable_baselines.bench import Monitor


C:\Users\kvnta\Anaconda3\lib\site-packages\dask\dataframe\utils.py:15: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



C:\Users\kvnta\Anaconda3\lib\site-packages\stable_baselines\__init__.py:33: UserWarning: stable-baselines is in maintenance mode, please use [Stable-Baselines3 (SB3)](https://github.com/DLR-RM/stable-baselines3) for an up-to-date version. You can find a [migration guide](https://stable-baselines3.readthedocs.io/en/master/guide/migration.html) in SB3 documentation.
  "stable-baselines is in maintenance mode, please use [Stable-Baselines3 (SB3)](https://github.com/DLR-RM/stable-baselines3) for an up-to-date version. You can find a [migration guide](https://stable-baselines3.readthedocs.io/en/master/guide/migration.html) in SB3 documentation."


In [2]:
env_id = 'SuperMarioBros-1-1-v0'
env = SubprocVecEnv([lambda: Monitor(wrapper(JoypadSpace(
    gym_super_mario_bros.make(env_id), RIGHT_ONLY)), 
    'monitor', allow_early_resets=True) for i in range(8)])

#env = DummyVecEnv([lambda: Monitor(wrapper(JoypadSpace(
#    gym_super_mario_bros.make(env_id), RIGHT_ONLY)), 
#    'monitor', allow_early_resets=True)])


# Parameters
#states = (84, 84, 4)
#actions = env.action_space.n

# Agent
#agent = DQNAgent(states=states, actions=actions, max_memory=100000, double_q=True)

In [3]:
# Evaluation Function
def evaluate(model, num_steps=1000):
    """
    Evaluate a RL agent
    :param model: (BaseRLModel object) the RL Agent
    :param num_steps: (int) number of timesteps to evaluate it
    :return: (float) Mean reward for the last 100 episodes
    """
    episode_rewards = [0.0]
    obs = env.reset()
    for i in range(num_steps):
        # _states are only useful when using LSTM policies
        action, _states = model.predict(obs)
        obs, reward, done, info = env.step(action)
        # Stats
        episode_rewards[-1] += reward
    obs = env.reset()
    episode_rewards.append(0.0)

    # Compute mean reward for the last 100 episodes
    mean_100ep_reward = np.mean(episode_rewards[-100:])
    print("Mean reward:", mean_100ep_reward, "Num episodes:", len(episode_rewards))

    return mean_100ep_reward

In [3]:
model = PPO2.load("ppo2-mario-CNNLNLSTM8-2000000.pkl", env, verbose=1)
from stable_baselines.common.evaluation import evaluate_policy
#mean_reward, std_reward = evaluate_policy(model, model.get_env(), n_eval_episodes=1)

C:\Users\kvnta\Anaconda3\lib\site-packages\cloudpickle\cloudpickle.py:704: UserWarning: A pickle file created using an old (<=1.4.1) version of cloudpicke is currently being loaded. This is not supported by cloudpickle and will break in cloudpickle 1.7
  "will break in cloudpickle 1.7", category=UserWarning






Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where





In [13]:
env.close()
del env

env = DummyVecEnv([lambda: Monitor(wrapper(JoypadSpace(
    gym_super_mario_bros.make(env_id), RIGHT_ONLY)), 
    'monitor', allow_early_resets=True)])

obs = env.reset()


_states = model.initial_state  # get the initial state vector for the recurrent network
actions = np.zeros(_states.shape[0])  # set all environment to not done
for _ in range(2000):
    zero_completed_obs = np.zeros((8,) + env.observation_space.shape)
    zero_completed_obs[0, :] = obs
    action, _states = model.predict(zero_completed_obs, deterministic=False)
    #action = action[0]
    if np.random.rand() < 0:
        action = np.random.randint(low=0, high=4, size=1)
    obs, rewards, dones, info = env.step(action)
    env.render()

In [6]:
obs = env.reset()
actions = []
for _ in range(10000):
    action, _states = model.predict(obs)
    if np.random.rand() < 0:
        action = np.random.randint(low=0, high=4, size=1)
    actions.append(action)
    obs, rewards, dones, info = env.step(action)
    env.render()

In [4]:
model = PPO2(MlpPolicy, env, verbose=1) 

# Random Agent, before training
mean_reward_before_train = evaluate(model, num_steps=1000)

# Train model
model.learn(total_timesteps=50000)

# Save model
model.save("ppo2-mario-50000")

# Random Agent, after training
mean_reward_after_train = evaluate(model, num_steps=1000)





Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Please use `layer.__call__` method instead.





Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where





KeyboardInterrupt: 